# 模型参数初始化

参数初始化的重要性  

初始化思路

初始化策略

为什么Transformer 除$\sqrt{d_k}$

## 参数初始化的重要性 
https://www.deeplearning.ai/ai-notes/initialization/

https://towardsdatascience.com/what-is-weight-initialization-in-neural-nets-and-why-it-matters-ec45398f99fa

参数设置过大或者过小都会导致模型不能训练或训练时间过长

## 初始化思路
思路： 输入数据和输出数据的模长保持不变

对于正交矩阵W: 

$\Vert Wx\Vert^2 = x^\top W^\top Wx = x^\top x =\Vert x\Vert^2$

对于全连接层： $y = Wx + b$, 全零初始化b, 并将W初始化为正交矩阵。


### 怎样得到正交矩阵？
推论1： 高维空间中的任意两个随机向量几乎都是垂直的。

推论2: 从$\mathcal{N}(0, 1/n)$中随机选取$n^2$个数，组成n*n的矩阵，这个矩阵近似为正交矩阵

In [5]:
import numpy as np

n = 100
W = np.random.randn(n, n) / np.sqrt(n)
X = np.dot(W.T, W)  # 矩阵乘以自身的转置
print(X)  # 看看是否接近单位阵
print('MSE', np.square(X - np.eye(n)).mean())  # 计算与单位阵的mse

[[ 1.23638913  0.15947432 -0.0455157  ... -0.08210088  0.0087467
  -0.17826778]
 [ 0.15947432  1.12888214  0.01165964 ... -0.15408815  0.12484842
   0.01831499]
 [-0.0455157   0.01165964  0.8616019  ...  0.06651855  0.00147615
  -0.03701675]
 ...
 [-0.08210088 -0.15408815  0.06651855 ...  1.11691626 -0.18314423
  -0.11051729]
 [ 0.0087467   0.12484842  0.00147615 ... -0.18314423  1.22016767
  -0.01922771]
 [-0.17826778  0.01831499 -0.03701675 ... -0.11051729 -0.01922771
   0.84414783]]
MSE 0.01023289441320619


推论一：$x_i, y_i$从正态分布采样
$$\mathbb{E}[x^\top y] = \mathbb{E}[\sum_i^n x_i y_i] = \sum_i^n \mathbb{E}[x_i y_i] = n\mathbb{E}[x_i] \mathbb{E}[y_i] = 0$$

推论二： 
因为$\mathbb{E}_{x\sim \mathcal{N}(0, 1)} [x^2] = var(\mathcal{N}(0, 1)) = 1$

所以，从$\mathcal{N}(0, 1)$中采样的n维向量模长期望是$\sqrt{n}$ ， 为了构造正交矩阵，从$\mathcal{N}(0, 1/n) $中采样。

### 怎么初始化？

考虑全连接层 :$y = Wx + b, W\in R^{m\times n}$

全零初始化b， 以随机正交矩阵初始化。

当$m\ge n$时， 从$\mathcal{N}(0, 1/m)$中采样的矩阵，近似满足$W^\top W = I$, 

若$m < n$, W不是行满秩矩阵，不存在$W^\top W = I$

## 初始化策略
上文没有考虑激活函数，常用的初始化策略有

### LeCun初始化
$\mathcal{N}(0, 1/m)$

### He 初始化
$\mathcal{N}(0, 2/m)$

针对激活函数为relu的情况， 有一半的输出被置0， 所以模长会减小为$1/\sqrt{2}$倍

### Xavier 初始化
$\mathcal{N} (0, 2/(m+ n))$

对比： https://www.deeplearning.ai/ai-notes/initialization/

## Transformer

 
$$\mathrm{Attention} (Q, K, V) = \mathrm{softmax}(\frac{QK^T}{\sqrt{d_k}})V $$

为什么除$\sqrt{d_k}$?

假设$q_i, k_i \sim \mathcal{N}(0, 1)$, 

$\mathbb{E}[(q^\top k)^2] = \mathbb{E}[(\sum_{i=1}^d q_i k_i)^2] = \mathbb{E} [(\sum_i q_i k_i)(\sum_j q_j k_j)] = \mathbb{E} [\sum_{i, j}(q_i q_j)(k_i k_j)] = \sum_{i,j} \mathbb{E}[q_iq_j] \mathbb{E} [k_i k_j] = d $

大致可以认为，内积后的数值在$-3\sqrt{d}$到$3\sqrt{d}$之间， $d=64, \sqrt{d}=8$当$3\sqrt{d}$过大时，softmax的效果会类似arg max, 出现梯度消失的问题